# Pandas – Iterace přes DataFrame

## Úvod do iterace

Iterace přes DataFrame umožňuje procházet data řádek po řádku nebo sloupec po sloupci. Ačkoli Pandas nabízí výkonné vektorizované operace, někdy je iterace nezbytná pro složité úlohy vyžadující individuální zpracování každého řádku.

### Kdy iteraci používat

| Scénář                      | Vhodnost iterace | Alternativa            |
|----------------------------|------------------|------------------------|
| Složité podmíněné logiky   | Vhodné           | `apply()`              |
| Přístup k indexu během operace | Vhodné      | `reset_index()` + vektorizace |
| Jednoduché matematické operace | Nevhodné     | Vektorizované operace  |
| Transformace dat           | Částečně vhodné  | `apply()`, `map()`     |

## Přehled metod iterace

| Metoda           | Popis               | Výstup         | Rychlost   |
|------------------|---------------------|----------------|------------|
| `df.iterrows()`  | Iterace přes řádky  | Index + Series | Pomalá     |
| `df.itertuples()`| Iterace přes řádky  | Named tuple    | Rychlejší  |
| `df.items()`     | Iterace přes sloupce| Název + Series | Střední    |

## Iterace přes řádky – `iterrows()`

In [1]:
import pandas as pd

data_ukazka = {
    'rok': [2020, 2020, 2021, 2021, 2022],
    'mesic': [1, 6, 3, 9, 12],
    'srazky': [15.2, 45.8, 23.1, 67.3, 12.9]
}
df_ukazka = pd.DataFrame(data_ukazka)

for index, row in df_ukazka.iterrows():
    print(f"Index: {index}")
    print(f"Rok: {row['rok']}, Měsíc: {row['mesic']}, Srážky: {row['srazky']}")

Index: 0
Rok: 2020.0, Měsíc: 1.0, Srážky: 15.2
Index: 1
Rok: 2020.0, Měsíc: 6.0, Srážky: 45.8
Index: 2
Rok: 2021.0, Měsíc: 3.0, Srážky: 23.1
Index: 3
Rok: 2021.0, Měsíc: 9.0, Srážky: 67.3
Index: 4
Rok: 2022.0, Měsíc: 12.0, Srážky: 12.9


## Iterace přes řádky – `itertuples()`

In [2]:
data_tuples = {
    'rok': [2020, 2021, 2022],
    'mesic': [6, 7, 8],
    'srazky': [45.8, 67.3, 52.1]
}
df_tuples = pd.DataFrame(data_tuples)

for row_tuple in df_tuples.itertuples():
    print(f"Index: {row_tuple.Index}")
    print(f"Rok: {row_tuple.rok}, Měsíc: {row_tuple.mesic}, Srážky: {row_tuple.srazky}")

Index: 0
Rok: 2020, Měsíc: 6, Srážky: 45.8
Index: 1
Rok: 2021, Měsíc: 7, Srážky: 67.3
Index: 2
Rok: 2022, Měsíc: 8, Srážky: 52.1


## Iterace přes sloupce – `items()`

In [3]:
data_sloupce = {
    'rok': [2020, 2021, 2022],
    'mesic': [6, 7, 8],
    'srazky': [45.8, 67.3, 52.1],
    'teplota': [22.5, 24.1, 23.8]
}
df_sloupce = pd.DataFrame(data_sloupce)

for nazev_sloupce, data_sloupce in df_sloupce.items():
    print(f"Sloupec: {nazev_sloupce}")
    print(f"Typ dat: {data_sloupce.dtype}")
    print(f"Počet hodnot: {len(data_sloupce)}")
    print(f"Chybějící hodnoty: {data_sloupce.isna().sum()}")

Sloupec: rok
Typ dat: int64
Počet hodnot: 3
Chybějící hodnoty: 0
Sloupec: mesic
Typ dat: int64
Počet hodnot: 3
Chybějící hodnoty: 0
Sloupec: srazky
Typ dat: float64
Počet hodnot: 3
Chybějící hodnoty: 0
Sloupec: teplota
Typ dat: float64
Počet hodnot: 3
Chybějící hodnoty: 0


## Iterace přes index

In [4]:
data_index = {
    'rok': [2020, 2021, 2022, 2020, 2021],
    'mesic': [6, 7, 8, 9, 10],
    'srazky': [45.8, 67.3, 52.1, 38.9, 29.4]
}
df_index = pd.DataFrame(data_index)

unikatni_roky = set()

for index in df_index.index:
    rok = df_index.loc[index, 'rok']
    unikatni_roky.add(rok)

print(sorted(unikatni_roky))

[2020, 2021, 2022]


## Iterace přes hodnoty – `values`

In [5]:
data_hodnoty = {
    'rok': [2020, 2021, 2022],
    'mesic': [6, 7, 8],
    'srazky': [145.8, 67.3, 152.1]
}
df_hodnoty = pd.DataFrame(data_hodnoty)

vysoke_srazky = []

for hodnoty_radku in df_hodnoty.values:
    rok, mesic, srazky = hodnoty_radku
    if srazky > 100:
        vysoke_srazky.append({
            'rok': rok,
            'mesic': mesic,
            'srazky': srazky
        })

df_vysoke = pd.DataFrame(vysoke_srazky)
print(df_vysoke)

      rok  mesic  srazky
0  2020.0    6.0   145.8
1  2022.0    8.0   152.1


## Alternativy k iteraci – `apply()`

In [6]:
data_apply = {
    'rok': [2020, 2021, 2022, 2020, 2021],
    'mesic': [6, 12, 8, 1, 10],
    'srazky': [85.8, 5.3, 125.1, 15.9, 45.4]
}
df_apply = pd.DataFrame(data_apply)

def klasifikace_pocasi(row):
    if row['mesic'] in [6, 7, 8] and row['srazky'] > 100:
        return 'bourkove_leto'
    elif row['mesic'] in [12, 1, 2] and row['srazky'] < 5:
        return 'suche_zimni_obdobi'
    elif row['srazky'] > 80:
        return 'vysoke_srazky'
    else:
        return 'standardni_pocasi'

df_apply['typ_pocasi'] = df_apply.apply(klasifikace_pocasi, axis=1)
print(df_apply)

    rok  mesic  srazky         typ_pocasi
0  2020      6    85.8      vysoke_srazky
1  2021     12     5.3  standardni_pocasi
2  2022      8   125.1      bourkove_leto
3  2020      1    15.9  standardni_pocasi
4  2021     10    45.4  standardni_pocasi
